This notebook will use GOUDA datasets and services created for working with large datasets:

https://github.com/bio-guoda/guoda-datasets


A well-formed Spark dataframe of iDigBio data will be loaded and we will extract the data that is of interest to our group.

In [1]:
idb_df = sqlContext.read.parquet("/guoda/data/idigbio-20181020T023317.parquet")
idb_df.count()

115173471

Our iDigBio dataframe has ~115 million observations. The schema looks like this: 

In [2]:
idb_df.printSchema()

root
 |-- barcodevalue: string (nullable = true)
 |-- basisofrecord: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- canonicalname: string (nullable = true)
 |-- catalognumber: string (nullable = true)
 |-- class: string (nullable = true)
 |-- collectioncode: string (nullable = true)
 |-- collectionid: string (nullable = true)
 |-- collectionname: string (nullable = true)
 |-- collector: string (nullable = true)
 |-- commonname: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- coordinateuncertainty: float (nullable = true)
 |-- country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- coreid: string (nullable = true)
 |    |-- dc:rights: string (nullable = true)
 |    |-- dcterms:accessRights: string (nullable = true)
 |    |-- dcterms:bibliographicCitation: string (nullable = true)
 |    |-- dcterms:language: string (nullable = true)
 |    |-- d

List of the columns that we want to summarize. These are the columns of current interest to the DwC group at the moment. Many columns have unique identifiers, free text, or already normalized vocab so it doesn't make sense to summarize everything for the purpose of trying to do everything at this point and a future more formal data product might.


In [3]:
columns = [
        "recordset",
        "data.dwc:kingdom",
        "data.dwc:phylum",
        "data.dwc:class",
        "data.dwc:order",
        "data.dwc:family",
        "data.dwc:genus",
        "data.dwc:specificEpithet",
        "data.dwc:taxonRank",
        "data.dwc:scientificName",
        "data.dwc:scientificNameAuthorship",
        "data.dwc:typeStatus",
        "data.dwc:identifiedBy",

        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "specificepithet",
        "taxonrank",
        "scientificname",
        "typestatus"

        ]

In [4]:
# Some fancy code from the internet. Note this transform could also be done 
# by manually typing out a bunch of unions, one for each column. For columns
# with highly repetitious values this is a reasonable all-at-once approach.
# For a more general solution, looping over the columns will result in much
# smaller data sizes and is more appropriate for a long term data product.

from pyspark.sql.functions import array, col, explode, struct, lit


def to_long(df, by):

    # Filter dtypes and split into column names and type description
    cols, dtypes = zip(*((c, t) for (c, t) in df.dtypes if c not in by))
    # Spark SQL supports only homogeneous columns
    assert len(set(dtypes)) == 1, "All columns have to be of the same type"

    # Create and explode an array of (column_name, column_value) structs
    kvs = explode(array([
      struct(lit(c).alias("key"), col(c).alias("val")) for c in cols
    ])).alias("kvs")

    return df.select(by + [kvs]).select(by + ["kvs.key", "kvs.val"])


We can now create a dataframe of key value pairs of our columns and their recordset identifier

In [5]:
k_v_pairs = to_long(idb_df.select(columns), ["recordset"])
k_v_pairs.head(10)

[Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:kingdom', val='Animalia'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:phylum', val='Arthropoda'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:class', val='Insecta'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:order', val='Coleoptera'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:family', val='Staphylinidae'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:genus', val='Philonthus'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:specificEpithet', val=None),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:taxonRank', val=None),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:scientificName', val='Philonthus'),
 Row(recordset='568e209f-d072-4fd6-8b64-27954b0fd731', key='dwc:scientificNameAuthorship', val=None)]

We can get an idea of the size of the data we are looking at by the number of key value pairs that are returned. This number should be close to the number of colums multiplied by the number of rows...

In [6]:
k_v_pairs.count()

2533816362

If we summarize these pairs by the number of times they occur within each column, we can then use this unique value dataframe for further analysis. 

In [9]:
index = (k_v_pairs
         .groupBy(k_v_pairs.recordset, k_v_pairs.key, k_v_pairs.val)
         .count()
         )
index.head(10)

[Row(recordset='e39f6dee-f2cf-4eff-afc9-4600cafe660c', key='dwc:order', val='Leucodontales', count=2398),
 Row(recordset='5ab348ab-439a-4697-925c-d6abe0c09b92', key='dwc:class', val='Reptilia', count=44078),
 Row(recordset='9368e302-f8e7-4714-aed4-db2faa861e5c', key='dwc:genus', val='Platismatia', count=929),
 Row(recordset='515cab85-98d0-4a23-9544-9e7b1c51f5a6', key='dwc:scientificName', val='Psarocolius wagleri', count=28),
 Row(recordset='616857b7-f952-44ef-9b6f-576dc1e65b51', key='family', val='piperaceae', count=15705),
 Row(recordset='c38b867b-05f3-4733-802e-d8d2d3324f84', key='order', val='scorpaeniformes', count=6758),
 Row(recordset='6bb853ab-e8ea-43b1-bd83-47318fc4c345', key='scientificname', val='euconulus fulvus', count=378),
 Row(recordset='09edf7d2-e68e-4a42-93da-762f86bb814f', key='dwc:kingdom', val='Plantae', count=33863),
 Row(recordset='437826f3-69f9-43d9-b3c3-c0de0e26cd88', key='dwc:family', val='Macrobiotidae', count=6171),
 Row(recordset='f630e3ea-697f-404a-8683-b8

In [10]:
index.count()

32245119

Subset to only paleo collections or collections with paleo specimens

In [11]:
paleoRecordsets = [
"6c6f34ed-58a4-4ba2-b9c7-34524f79a349",
"137ed4cd-5172-45a5-acdb-8e1de9a64e32",
"5ab348ab-439a-4697-925c-d6abe0c09b92",
"f9a33279-d6ba-41c7-a511-ef6adfcb6e20",
"95ecb448-3c1f-4145-8565-4f6d51beb62c", 
"271a9ce9-c6d3-4b63-a722-cb0adc48863f",
"1ba0bbad-28a7-4c50-8992-a028f79d1dc5",
"62c310ac-e1ff-47bc-860d-0471a84ed0d3",
"271a9ce9-c6d3-4b63-a722-cb0adc48863f",
"b1f0612a-bc21-424f-b9c1-3bba69ad4f54",
"7b0809fb-fd62-4733-8f40-74ceb04cbcac",
"7ae4d15d-62e2-459b-842a-446f921b9d3f",
"1ebb0c8e-31f2-4564-b75d-65196bee4f09",
"71b8ffab-444e-43f9-9a9c-5c42b0eaa5eb",
"d621e959-2633-4ec1-a2a2-5d97cd818b47",
"0220907a-0463-4ae0-8a0b-77f5e80fff40",
"84c24d87-e4ad-4165-8e86-5ae1a249c196",
"b26fa674-6300-4ea0-a8e3-fc0ce32b5226",
"41b119de-f745-482d-be42-a0155bc76e5d",
"0e162e0a-bf3e-4710-9357-44258ca12abb",
"667c2736-bcd3-4a6a-abf4-db5d2dc815c4",
"2ec3b31e-c86b-4ce9-b265-77c8c3f9643c",
"81316846-80cb-4913-8941-b31537761eb0", 
"1c8d18f4-5af2-4d86-98d2-8a5ed06456e2", 
"7c2c5cdc-80e6-49d5-8e95-08fc7da0a370",
"db3181c9-48dd-489f-96ab-a5888f5a938c", 
"4dfb5828-3653-4604-ac00-db1e1da98b02", 
"7757c07f-18fd-45c2-84cc-60bd3742e100",
"b7a79601-c07b-46d5-bd09-d4472b0d9431",
"9e66257f-21a9-491a-ac23-06b7b62ceeb7",
"d11f19ae-e946-4a0e-83e5-2052ae8cca62",
"ba77d411-4179-4dbd-b6c1-39b8a71ae795",
"e27f0218-47e0-41bc-9086-9d9169096e90",
"879d475f-4b76-4d18-8cf6-a7e5a6d44926",
"a2beb85e-f2b8-4366-8b3b-e5c5cc117aaf",
"2c2cc29c-3572-4568-a129-c8cbec34ccbe",
"a2a7754b-2346-496d-b681-eb754ef32b9e",
"0d05a365-36e8-4150-a350-23ed33f79b17",
"5082e6c8-8f5b-4bf6-a930-e3e6de7bf6fb",
"bf049384-ffe2-4418-a1a3-fc5552ba850f"
]

In [12]:
test = index[index['recordset'].isin(paleoRecordsets)]
test.head(10)

[Row(recordset='137ed4cd-5172-45a5-acdb-8e1de9a64e32', key='scientificname', val='spirifer alatus', count=47),
 Row(recordset='5ab348ab-439a-4697-925c-d6abe0c09b92', key='dwc:class', val='Reptilia', count=44078),
 Row(recordset='81316846-80cb-4913-8941-b31537761eb0', key='taxonrank', val='species', count=8752),
 Row(recordset='5ab348ab-439a-4697-925c-d6abe0c09b92', key='dwc:order', val='Mesosauria', count=11),
 Row(recordset='5ab348ab-439a-4697-925c-d6abe0c09b92', key='dwc:scientificName', val='Prolagus oeningensis', count=10),
 Row(recordset='95ecb448-3c1f-4145-8565-4f6d51beb62c', key='dwc:scientificName', val='Juresania?', count=91),
 Row(recordset='137ed4cd-5172-45a5-acdb-8e1de9a64e32', key='family', val='iridinidae', count=611),
 Row(recordset='137ed4cd-5172-45a5-acdb-8e1de9a64e32', key='family', val='solenopleuridae', count=375),
 Row(recordset='1ebb0c8e-31f2-4564-b75d-65196bee4f09', key='scientificname', val='leuciscus papyraceus', count=2),
 Row(recordset='71b8ffab-444e-43f9-9a9

In [13]:
test.count()

1982376

In [14]:
(test
 .write
 .parquet("/home/kevinlove/paleo/outputs/paleo-TAXON-values-20182811.parquet")
)

In [15]:
(test
 .write
 .format("com.databricks.spark.csv")
 .option("header", "false")
 .save("/home/kevinlove/paleo/outputs/paleo-TAXON-value-20182811.csv")
)

We can continue to work with these data here or access them via the filesystem by saving a copy locally

In [16]:
from hdfs import InsecureClient
client = InsecureClient('http://mesos01.acis.ufl.edu:50070/', user='kevinlove')
client.download('/home/kevinlove/paleo/outputs/paleo-TAXON-value-20182811.csv', '/home/kevinlove/paleo/outputs/paleo-TAXON-values-20182811', n_threads=5)

'/home/kevinlove/paleo/outputs/paleo-TAXON-values-20182811'